In [ ]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.insert(0, '../JKBio/')
import Helper as helper
import igv
import numpy as np
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

In [ ]:
! gcsfuse --only-dir Chip_AML amlproject data/seqs

In [ ]:
!fusermount -u data/seqs

In [ ]:
! ls data/seqs/GEO

## finding all the relevant files for chipseq processing

In [ ]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip

In [ ]:
a

In [ ]:
rename1 = {'gs://amlproject/Chip_AML/Chip/20180709_1_MP5773_S1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp631.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_3_MP5773_S3.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp633.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_5_MP5773_S5.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp635.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_6_MP5773_S6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp636.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-11_MP7213-2of2_S4_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp748.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-13_MP7213-2of2_S5_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp750.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-14_MP7213-2of2_S6_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp751.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-2_MP7213-1of2_S2_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp739.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-4_MP7213-1of2_S4_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp741.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-5_MP7213-1of2_S5_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp742.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-6_MP7213-1of2_S6_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp743.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-8_MP7213-2of2_S1_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp745.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_10_MB7326-2of2_S5.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp761.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_11_MB7326-2of2_S6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp762.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_9_MB7326-2of2_S4.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp760.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D0-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp8.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp14.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9_CEBPA_m.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp27.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9_INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp35.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP29.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp4.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP30.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp5.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP31.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp6.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp24.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2_CEBPA_m.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp41.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2_INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp33.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_10-D8-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp156.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_12-MV411-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp115.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_15-MV411-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp118.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_18-D8-MED1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp160.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_19-M6-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp122.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_20-M6-RUNX1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp123.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_21-M6-MYB.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp124.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_22-M6-ETV6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp125.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_24-M6-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp171.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_25-M6-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp127.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_26-M6-MED1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp128.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_27-MV411-RUNX1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp129.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_27-OCIAML2-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp168.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_31.M6.INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp98.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_41-M6-IGG.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp143.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_6-M6-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp109.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_7-D8-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp153.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_8-D8-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp154.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_9-D8-MYB.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp155.fastq.gz'
}

## looking for duplicates

In [ ]:
import re
mps=[]
for i in a:
    res = re.search('mp(\d+)', i)
    if res is not None:
        mps.append(res.group(0))

In [ ]:
mps.sort()

In [ ]:
mps

In [ ]:
duplicates=[ 'gs://amlproject/Chip_AML/Chip/Sample_mp582-MV411-MEIS1-r1.fastq.gz',
'gs://amlproject/Chip_AML/Chip/Sample_mp587-MV411-IKZF1-r1.fastq.gz',
,
'gs://amlproject/Chip_AML/Chip/mp88-MV411-H3K27ac-r2.fastq.gz']

In [ ]:
for val in duplicates:
    os.system('gsutil rm '+val)

In [ ]:
paired_end_tomove= ['gs://amlproject/Chip_AML/Chip/mp304-D9-PU1-r2_S1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp305-SKNO1-PU1-r1_S2_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp320-MV411-ZEB2-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp319-MV411-FLI1-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp308-SKNO1-RUNX1-RUNX1T1-r1_S5_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp321-MV411-GFI1-r4_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp324-MV411-MEF2D-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp299-MV411-CEBPA-r3_S7_R2.fastq.gz'
 'gs://amlproject/Chip_AML/Chip/mp325-MV411-SP1-r2_R2.fastq.gz']

In [ ]:
for val in paired_end_tomove:
    os.system('gsutil mv '+val+' gs://amlproject/Chip_AML/Chip/paired_end/')

## renamings

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
## IT SHOULD NOT CAUSE ANY PROBLEM TO CP R2 SINCE IT DOES NOT EXIST 
#(will throw an error and skip)
import re
import time
mps=[]
for i in a:
    res = re.search('mp(\d+)', i)
    if res is not None:
        print('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')
        time.sleep(2)
        os.system('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')

In [ ]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

In [ ]:
# EXCEPT these two... TO RETRIEVE
gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz
gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R1.fastq.gz

In [ ]:
stored = !gsutil ls -a gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz
stored=stored[0]
! gsutil cp $stored gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz

In [ ]:
a

In [ ]:
#continuing
a[85]

In [ ]:
import re
import time
mps=[]
for i in a[86:]:
    res = re.search('mp(\d+)', i)
    if res is not None:
        print('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')
        time.sleep(2)
        os.system('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')

## adding metadata

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')

In [ ]:
url="https://docs.google.com/spreadsheets/d/14onj_lD6WrjSEsE5dWJid6lGdJ81O6qpcetP5_Z0FxQ"
gsheets = sheets.get(url).sheets[1].to_frame()

In [ ]:
gsheets

In [ ]:
gsheets = gsheets.set_index('Unique sample ID')

In [ ]:
a = !gsutil -m ls gs://amlproject/Chip_AML/Chip/
a

In [ ]:
data = pd.DataFrame({'fastq_1':a[:-1]}, index=[i.split('/')[-1].split('.')[0] for i in a[:-1]])

In [ ]:
data[['quality','Protein','Replicate No','Cell line ID']] = gsheets[
    ['quality','Protein','Replicate No','Cell line ID']]

In [ ]:
data['fastq_2']= np.NaN

In [ ]:
data = data.rename(columns={'Protein':'antibody','Cell line ID':'group','Replicate No':'replicate'})

In [ ]:
data.loc[data[data.antibody.isin(['INPUT','IGG'])].index,'group'] = \
data[data.antibody.isin(['INPUT','IGG'])].antibody +'_'+\
data[data.antibody.isin(['INPUT','IGG'])].group

In [ ]:
data.loc[data[~data.antibody.isin(['INPUT','IGG'])].index,'control'] = \
'INPUT_'+\
data[~data.antibody.isin(['INPUT','IGG'])].group

In [ ]:
data

In [ ]:
set(data.control)-set(data.group)

In [ ]:
withoutinput = ["HEL",
                "SHI1",
                "EOL1",
                "F36P"]

In [ ]:
# requesting them and downloading them...

In [ ]:
rename = {'gs://amlproject/Chip_AML/Chip/1_MP5835-1of4_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp643.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_17-HL60-INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp120.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_33.U937.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp100.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_34.NOMO1.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp101.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_35.UT7.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp102.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp309-KG1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp309.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp310-Kasumi1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp310.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp311-MOLM13-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp311.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp312-NB4-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp312.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp313-TF1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp313.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp314-OCIAML3-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp314.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp315-MONOMAC3-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp315.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp326-DFAM71927V3-INPUT-r1_R1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp326.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/mp609-MONOMAC1-INPUT-r1_S4.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp609.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp610-HEL9217-INPUT-r1_S5.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp610.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp611-P31FUJ-INPUT-r1_S6.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp611.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp612-PLB985-INPUT-r1_S7.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp612.fastq.gz',
          
 'gs://amlproject/Chip_AML/Chip/1_MP5835-1of4_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/paired_end/mp643-1_MP5835-1of4_S1_R2_001.fastq.gz",          
'gs://amlproject/Chip_AML/Chip/Sample_mp326-DFAM71927V3-INPUT-r1_R2.fastq.gz':"gs://amlproject/Chip_AML/Chip/paired_end/mp326-DFAM71927V3-INPUT-r1_R2.fastq.gz"}

In [ ]:
for k,v in rename.items():
    ! gsutil mv $k $v

In [ ]:
data = data.drop(data[data.group.isin(withoutinput)].index)

In [ ]:
data.loc[data[~data.antibody.isin(['INPUT','IGG'])].index,'group'] = \
data[~data.antibody.isin(['INPUT','IGG'])].group + '_' + \
data[~data.antibody.isin(['INPUT','IGG'])].replicate.astype(str)

In [ ]:
data[data.antibody.isin(['INPUT','IGG'])].antibody = np.nan

In [ ]:
data['replicate']=1

In [ ]:
data

In [ ]:
data[data.columns[[4,3,0,5,2,6]]].to_csv('nextflow/design.csv', index=False)

In [ ]:
! export GOOGLE_APPLICATION_CREDENTIALS=~/jkobject-b6f1adaffcb8.json
! export NXF_MODE=google
! nextflow info

In [ ]:
! nextflow cloud create 'jkcluster' -c 40

In [ ]:
! nextflow run nf-core/chipseq -c "nextflow/nextflow.config" \
--singleEnd \
--seq_center 'DFCI' \
--email 'amlproject@gmail.com' \
--bucket-dir 'gs://amlproject/Chip_AML/nextflow/CHIPprocess_2/' \
--keyfile '~/amlproject-b6f1adaffcb8.json' \
--projectname 'amlproject' \
--zone 'us-east1-b' \
--skipDiffAnalysis \
--narrowPeak \
--design "nextflow/design.csv" \
--genome "GRCh38" \
--profile 'gcp' \
--resume \
--skipPreseq \
--max_cpus 8

In [ ]:
! nextflow cloud shutdown jkcluster

In [ ]:
gsheets[gsheets['Cell line ID'].isin([
    "MOLM13",
"MONOMAC1",
"MONOMAC6",
"NB4",
"UT7",
"KG1",
"U937",
"P31FUJ",
"HL60",
"PLB985",
"OCIAML3",
"HEL9217",
"Kasumi1",
"NOMO1",
"TF1",
"SHI1",
"DFAM71927V3",
"EOL1",
"F36P"
])][gsheets.Protein=='INPUT'].index

### need to use juicer
https://github.com/aidenlab/juicer/wiki/Data-Extraction

https://github.com/aidenlab/straw/wiki/Python

https://github.com/aidenlab/juicer/wiki/Data

https://github.com/aidenlab/juicer

In [ ]:
! pip install hic-straw

In [ ]:
import straw

In [ ]:
result = straw.straw('KR', 
                     'data/GEO/MV411_H3K27ac_all_rep_duplicate_removed_allValidPairs.hic',
                     'X', 'X', 'BP', 5000)

In [ ]:
len(result[1])

In [ ]:
! ls data/GEO